In [1]:
import requestcompletion as rc

from dotenv import load_dotenv
from enum import Enum
from typing import Tuple, List
_ = load_dotenv()

c:\Users\Logan\Documents\rc\.venv\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:

# in this cell I am making some mock function calls that you can imagine would be API calls if you were looking for an accurate system
class WeatherType(Enum):
    SUNNY = 'sunny'
    RAINY = 'rainy'
    CLOUDY = 'cloudy'
    SNOWY = 'snowy'
    WINDY = 'windy'
    
class LocationNotFoundException(Exception):
    pass

async def get_current_weather(location: str) -> Tuple[float, WeatherType]:
    location_weather_map = {
        "New York": (75.0, WeatherType.SUNNY),
        "Los Angeles": (90.0, WeatherType.CLOUDY),
        "Chicago": (65.0, WeatherType.RAINY),
        "Miami": (85.0, WeatherType.WINDY),
        "Seattle": (55.0, WeatherType.SNOWY),
        "Orlando": (80.0, WeatherType.SUNNY),
        "Denver": (70.0, WeatherType.CLOUDY),
        "San Francisco": (75.0, WeatherType.RAINY),
        "Boston": (65.0, WeatherType.WINDY),
        "Las Vegas": (90.0, WeatherType.SNOWY),
        "Vancouver": (55.0, WeatherType.SUNNY),
        "Toronto": (80.0, WeatherType.CLOUDY),
        "Portland": (55.0, WeatherType.RAINY),
        "Austin": (90.0, WeatherType.WINDY),
        "Houston": (85.0, WeatherType.SNOWY),
        "Springfield": (65.0, WeatherType.SUNNY),
    }
    
    if location not in location_weather_map:
        raise LocationNotFoundException(f"Location {location} not found")
    
    return location_weather_map[location]

async def get_weekly_forecast(location: str) -> List[Tuple[float, WeatherType]]:
    location_forecast_map = {
        "New York": [
            (75.0, WeatherType.SUNNY),
            (70.0, WeatherType.CLOUDY),
            (65.0, WeatherType.RAINY),
            (60.0, WeatherType.WINDY),
            (55.0, WeatherType.SNOWY),
            (50.0, WeatherType.SUNNY),
            (45.0, WeatherType.CLOUDY),
        ],
        "Los Angeles": [
            (90.0, WeatherType.CLOUDY),
            (85.0, WeatherType.WINDY),
            (80.0, WeatherType.SNOWY),
            (75.0, WeatherType.SUNNY),
            (70.0, WeatherType.CLOUDY),
            (65.0, WeatherType.RAINY),
            (60.0, WeatherType.WINDY),
        ],
        "Chicago": [
            (65.0, WeatherType.RAINY),
            (60.0, WeatherType.WINDY),
            (55.0, WeatherType.SNOWY),
            (50.0, WeatherType.SUNNY),
            (45.0, WeatherType.CLOUDY),
            (40.0, WeatherType.RAINY),
            (35.0, WeatherType.WINDY),
        ],
        "Miami": [
            (85.0, WeatherType.WINDY),
            (80.0, WeatherType.SNOWY),
            (75.0, WeatherType.SUNNY),
            (70.0, WeatherType.CLOUDY),
            (65.0, WeatherType.RAINY),
            (60.0, WeatherType.WINDY),
            (55.0, WeatherType.SNOWY),
        ],
        "Seattle": [
            (55.0, WeatherType.SNOWY),
            (50.0, WeatherType.SUNNY),
            (45.0, WeatherType.CLOUDY),
            (40.0, WeatherType.RAINY),
            (35.0, WeatherType.WINDY),
            (30.0, WeatherType.SNOWY),
            (25.0, WeatherType.SUNNY),
        ],
        "Orlando": [
            (80.0, WeatherType.SUNNY),
            (75.0, WeatherType.CLOUDY),
            (70.0, WeatherType.RAINY),
            (65.0, WeatherType.WINDY),
            (60.0, WeatherType.SNOWY),
            (55.0, WeatherType.SUNNY),
            (50.0, WeatherType.CLOUDY),
        ],
        "Denver": [
            (70.0, WeatherType.CLOUDY),
            (65.0, WeatherType.RAINY),
            (60.0, WeatherType.WINDY),
            (55.0, WeatherType.SNOWY),
            (50.0, WeatherType.SUNNY),
            (45.0, WeatherType.CLOUDY),
            (40.0, WeatherType.RAINY),
        ],
        "San Francisco": [
            (75.0, WeatherType.RAINY),
            (70.0, WeatherType.WINDY),
            (65.0, WeatherType.SNOWY),
            (60.0, WeatherType.SUNNY),
            (55.0, WeatherType.CLOUDY),
            (50.0, WeatherType.RAINY),
            (45.0, WeatherType.WINDY),
        ],
        "Boston": [
            (65.0, WeatherType.WINDY),
            (60.0, WeatherType.SNOWY),
            (55.0, WeatherType.SUNNY),
            (50.0, WeatherType.CLOUDY),
            (45.0, WeatherType.RAINY),
            (40.0, WeatherType.WINDY),
            (35.0, WeatherType.SNOWY),
        ],
        "Las Vegas": [
            (90.0, WeatherType.SNOWY),
            (85.0, WeatherType.SUNNY),
            (80.0, WeatherType.CLOUDY),
            (75.0, WeatherType.RAINY),
            (70.0, WeatherType.WINDY),
            (65.0, WeatherType.SNOWY),
            (60.0, WeatherType.SUNNY),
        ],
        "Vancouver": [
            (55.0, WeatherType.SUNNY),
            (50.0, WeatherType.CLOUDY),
            (45.0, WeatherType.RAINY),
            (40.0, WeatherType.WINDY),
            (35.0, WeatherType.SNOWY),
            (30.0, WeatherType.SUNNY),
            (25.0, WeatherType.CLOUDY),
        ],
        "Toronto": [
            (80.0, WeatherType.CLOUDY),
            (75.0, WeatherType.RAINY),
            (70.0, WeatherType.WINDY),
            (65.0, WeatherType.SNOWY),
            (60.0, WeatherType.SUNNY),
            (55.0, WeatherType.CLOUDY),
            (50.0, WeatherType.RAINY),
        ],
        "Portland": [
            (55.0, WeatherType.RAINY),
            (50.0, WeatherType.WINDY),
            (45.0, WeatherType.SNOWY),
            (40.0, WeatherType.SUNNY),
            (35.0, WeatherType.CLOUDY),
            (30.0, WeatherType.RAINY),
            (25.0, WeatherType.WINDY),
        ],
        "Austin": [
            (90.0, WeatherType.WINDY),
            (85.0, WeatherType.SNOWY),
            (80.0, WeatherType.SUNNY),
            (75.0, WeatherType.CLOUDY),
            (70.0, WeatherType.RAINY),
            (65.0, WeatherType.WINDY),
            (60.0, WeatherType.SNOWY),
        ],
        "Houston": [
            (85.0, WeatherType.SNOWY),
            (80.0, WeatherType.SUNNY),
            (75.0, WeatherType.CLOUDY),
            (70.0, WeatherType.RAINY),
            (65.0, WeatherType.WINDY),
            (60.0, WeatherType.SNOWY),
            (55.0, WeatherType.SUNNY),
        ],
        "Springfield": [
            (65.0, WeatherType.SUNNY),
            (60.0, WeatherType.CLOUDY),
            (55.0, WeatherType.RAINY),
            (50.0, WeatherType.WINDY),
            (45.0, WeatherType.SNOWY),
            (40.0, WeatherType.SUNNY),
            (35.0, WeatherType.CLOUDY),
        ],
    }
    
    if location not in location_forecast_map:
        raise LocationNotFoundException(f"Location {location} not found")
    
    return location_forecast_map[location]

async def get_long_term_forecast(location: str) -> str:
    location_long_term_map = {
        "New York": "The upcoming months will bring a mix of weather. Expect a lot of rain and some sun. The temperature will be mild.",
        "Los Angeles": "The upcoming months will be hot and dry. Expect a lot of sun and very little rain. The temperature will be hot.",
        "Chicago": "The upcoming months will be cold and wet. Expect a lot of snow and rain. The temperature will be cold.",
        "Miami": "The upcoming months will be hot and wet. Expect a lot of sun and rain. The temperature will be hot.",
        "Seattle": "The upcoming months will be cold and wet. Expect a lot of snow and rain. The temperature will be cold.",
        "Orlando": "The upcoming months will be hot and wet. Expect a lot of sun and rain. The temperature will be hot.",
        "Denver": "The upcoming months will be cold and dry. Expect a lot of snow and very little rain. The temperature will be cold.",
        "San Francisco": "The upcoming months will be cold and wet. Expect a lot of rain and very little sun. The temperature will be cold.",
        "Boston": "The upcoming months will be cold and wet. Expect a lot of snow and rain. The temperature will be cold.",
        "Las Vegas": "The upcoming months will be hot and dry. Expect a lot of sun and very little rain. The temperature will be hot.",
        "Vancouver": "The upcoming months will be cold and wet. Expect a lot of snow and rain. The temperature will be cold.",
        "Toronto": "The upcoming months will be cold and wet. Expect a lot of snow and rain. The temperature will be cold.",
        "Portland": "The upcoming months will be cold and wet. Expect a lot of snow and rain. The temperature will be cold.",
        "Austin": "The upcoming months will be hot and dry. Expect a lot of sun and very little rain. The temperature will be hot.",
        "Houston": "The upcoming months will be hot and wet. Expect a lot of sun and rain. The temperature will be hot.",
        "Springfield": "The upcoming months will be cold and wet. Expect a lot of snow and rain. The temperature will be cold.",
    }
    
    if location not in location_long_term_map:
        raise LocationNotFoundException(f"Location {location} not found")
    
    return location_long_term_map[location]

async def available_locations() -> List[str]:
    return [
        "New York",
        "Los Angeles",
        "Chicago",
        "Miami",
        "Seattle",
        "Orlando",
        "Denver",
        "San Francisco",
        "Boston",
        "Las Vegas",
        "Vancouver",
        "Toronto",
        "Portland",
        "Austin",
        "Houston",
        "Springfield",
    ]

AvailableLocations = rc.library.from_function(available_locations)
GetWeather = rc.library.from_function(get_current_weather)
GetWeeklyForecast = rc.library.from_function(get_weekly_forecast)
GetLongTermForecast = rc.library.from_function(get_long_term_forecast)



In [ ]:
system_prompt = "You are a helpful weather forecaster. You have access to the current weather, weekly forecast, and long-term forecast for various locations. You can provide this information to users who ask for it. You can also provide a list of available locations. Be creative and strong willed in your answers"
WeatherForecaster = rc.library.tool_call_llm(connected_nodes={AvailableLocations, GetWeather, GetWeeklyForecast, GetLongTermForecast}, model=rc.llm.OpenAILLM("gpt-4o"), system_message=rc.llm.SystemMessage(system_prompt))

Now we can run it

In [5]:
question = "Can you tell me about the current weather in New York and Boston?"  
mess_hist = rc.llm.MessageHistory([rc.llm.UserMessage(question)])
with rc.Runner() as runner:
    complete = await runner.run(
        WeatherForecaster, mess_hist,
    )

print(complete.answer)



assistant: The current weather in New York is sunny with a temperature of 75°F. In Boston, it's windy with a temperature of 65°F.
